# Beauty Retail Marketing Analysis

## Project Overview
This notebook analyzes **10,322 beauty products** from 1,090+ brands across 4 e-commerce platforms to understand customer behavior, pricing strategies, and market segmentation.

**Key Questions:**
- What drives customer satisfaction?
- How do pricing strategies affect ratings?
- Can we predict product ratings?
- What market segments exist?

**Dataset:** Multi-platform beauty product data (Amazon, Flipkart, Sephora, Ulta)

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.inspection import permutation_importance

# Display settings
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

## 1. Data Loading & Cleaning

In [ ]:
# Load dataset
df = pd.read_csv('../data/cleaned_dataset.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nColumn types:")
print(df.dtypes)
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Data quality summary
print("=== DATA QUALITY REPORT ===")
print(f"Total products: {len(df):,}")
print(f"Unique brands: {df['Brand'].nunique():,}")
print(f"Platforms: {', '.join(df['Website'].unique())}")
print(f"Product categories: {df['Category'].nunique()}")
print(f"Product forms: {df['Form'].nunique()}")
print(f"\nRating range: {df['Rating'].min():.1f} - {df['Rating'].max():.1f}")
print(f"Price range: ${df['Price ($)'].min():.0f} - ${df['Price ($)'].max():,.0f}")
print(f"\nMissing values:\n{df.isnull().sum()}")

## 2. Exploratory Data Analysis

### 2.1 Rating Distribution

In [ ]:
# Rating distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Histogram
axes[0].hist(df['Rating'], bins=25, edgecolor='black', alpha=0.7)
axes[0].axvline(df['Rating'].median(), color='red', linestyle='--', label=f'Median: {df["Rating"].median():.2f}')
axes[0].set_title('Rating Distribution', fontsize=14)
axes[0].set_xlabel('Rating')
axes[0].set_ylabel('Frequency')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Box plot
axes[1].boxplot(df['Rating'], vert=False, patch_artist=True,
                boxprops=dict(facecolor='skyblue'),
                medianprops=dict(color='red', linewidth=2))
axes[1].set_title('Rating Distribution (Box Plot)', fontsize=14)
axes[1].set_xlabel('Rating')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../visualizations/rating_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Rating Statistics:")
print(df['Rating'].describe())

### 2.2 Price vs Rating Analysis

In [ ]:
# Price vs Rating by Product Form
df_form = df.groupby('Form').agg({'Price ($)': 'mean', 'Rating': 'mean'}).reset_index()
correlation = df_form['Price ($)'].corr(df_form['Rating'])

plt.figure(figsize=(10, 6))
plt.scatter(df_form['Price ($)'], df_form['Rating'], s=100, alpha=0.6, edgecolors='black')
plt.title(f'Price vs Rating by Product Form (Correlation: {correlation:.3f})', fontsize=14)
plt.xlabel('Average Price ($)', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)
plt.grid(alpha=0.3)

# Add trend line
z = np.polyfit(df_form['Price ($)'], df_form['Rating'], 1)
p = np.poly1d(z)
plt.plot(df_form['Price ($)'], p(df_form['Price ($)']), "r--", alpha=0.8, label='Trend line')
plt.legend()

plt.tight_layout()
plt.savefig('../visualizations/price_vs_rating.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\nKey Finding: Price-Rating correlation = {correlation:.3f}")
print("Negative correlation suggests higher prices DON'T guarantee higher ratings!")

### 2.3 Platform Comparison

In [ ]:
# Website comparison
df_website = df.groupby('Website').agg({
    'Price ($)': 'mean',
    'Rating': 'mean',
    'Product name': 'count'
}).rename(columns={'Product name': 'Count'}).reset_index()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart - Price
axes[0].bar(df_website['Website'], df_website['Price ($)'], color='steelblue', edgecolor='black')
axes[0].set_title('Average Price by Platform', fontsize=14)
axes[0].set_ylabel('Average Price ($)', fontsize=12)
axes[0].grid(alpha=0.3, axis='y')

# Bar chart - Rating
axes[1].bar(df_website['Website'], df_website['Rating'], color='coral', edgecolor='black')
axes[1].set_title('Average Rating by Platform', fontsize=14)
axes[1].set_ylabel('Average Rating', fontsize=12)
axes[1].set_ylim(3.5, 4.5)
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('../visualizations/platform_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nPlatform Statistics:")
print(df_website)

## 3. Feature Engineering

Creating new features to improve model performance:

In [ ]:
# Load feature-engineered dataset
df_features = pd.read_csv('../data/data_with_features.csv')

print("Engineered Features:")
print("1. Price_Per_Rating: Price divided by rating (value metric)")
print("2. Is_Premium: Binary flag for products in top 25% price range")
print("3. Brand_Avg_Rating: Average rating for each brand")
print("4. Category_Avg_Price: Average price for each category")
print("\nSample data:")
df_features[['Price ($)', 'Rating', 'Price_Per_Rating', 'Is_Premium', 'Brand_Avg_Rating']].head(10)

In [ ]:
# Correlation heatmap
numeric_cols = ['Price ($)', 'Rating', 'Price_Per_Rating', 'Is_Premium', 
                'Brand_Avg_Rating', 'Category_Avg_Price']

plt.figure(figsize=(10, 8))
correlation_matrix = df_features[numeric_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0,
            fmt='.3f', square=True, linewidths=0.5)
plt.title('Feature Correlation Heatmap', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig('../visualizations/correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nKey Correlations with Rating:")
print(correlation_matrix['Rating'].sort_values(ascending=False))

## 4. Predictive Modeling

### 4.1 KNN Regression for Rating Prediction

In [ ]:
# Prepare data for modeling
le = LabelEncoder()

# Encode categorical variables
df_model = df_features.copy()
df_model['Website_Encoded'] = le.fit_transform(df_model['Website'])
df_model['Category_Encoded'] = le.fit_transform(df_model['Category'])
df_model['Subcategory_Encoded'] = le.fit_transform(df_model['Subcategory'])
df_model['Form_Encoded'] = le.fit_transform(df_model['Form'])

# Select features
feature_cols = ['Price ($)', 'Website_Encoded', 'Category_Encoded', 
                'Subcategory_Encoded', 'Form_Encoded', 'Price_Per_Rating',
                'Is_Premium', 'Brand_Avg_Rating']

X = df_model[feature_cols]
y = df_model['Rating']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Features: {feature_cols}")

In [ ]:
# Hyperparameter tuning with GridSearchCV
param_grid = {
    'n_neighbors': [5, 10, 15, 20, 25, 30],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

knn = KNeighborsRegressor()
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

print("Training KNN model with GridSearchCV...")
grid_search.fit(X_train_scaled, y_train)

print(f"\nBest parameters: {grid_search.best_params_}")
print(f"Best CV score (MSE): {-grid_search.best_score_:.4f}")

# Evaluate on test set
best_knn = grid_search.best_estimator_
test_score = best_knn.score(X_test_scaled, y_test)
predictions = best_knn.predict(X_test_scaled)
mse = np.mean((predictions - y_test) ** 2)

print(f"\nTest R² score: {test_score:.4f}")
print(f"Test MSE: {mse:.4f}")
print(f"Test RMSE: {np.sqrt(mse):.4f}")

In [ ]:
# Feature importance using permutation importance
perm_importance = permutation_importance(best_knn, X_test_scaled, y_test, 
                                         n_repeats=10, random_state=42)

# Create importance dataframe
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': perm_importance.importances_mean
}).sort_values('Importance', ascending=False)

# Plot
plt.figure(figsize=(10, 6))
plt.barh(importance_df['Feature'], importance_df['Importance'], color='teal', edgecolor='black')
plt.xlabel('Importance Score', fontsize=12)
plt.title('Feature Importance (Permutation)', fontsize=14)
plt.gca().invert_yaxis()
plt.grid(alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('../visualizations/feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nFeature Importance Rankings:")
print(importance_df.to_string(index=False))

## 5. Customer Segmentation with K-Means

### 5.1 Finding Optimal Clusters

In [ ]:
# Prepare clustering data
cluster_features = df_features[['Price ($)', 'Rating']].dropna()
scaler_cluster = StandardScaler()
cluster_scaled = scaler_cluster.fit_transform(cluster_features)

# Elbow method
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(cluster_scaled)
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters (k)', fontsize=12)
plt.ylabel('Inertia (Within-cluster sum of squares)', fontsize=12)
plt.title('Elbow Method for Optimal k', fontsize=14)
plt.grid(alpha=0.3)
plt.axvline(x=4, color='red', linestyle='--', label='Optimal k=4')
plt.legend()
plt.tight_layout()
plt.savefig('../visualizations/optimal_k.png', dpi=300, bbox_inches='tight')
plt.show()

print("Inertia values for different k:")
for k, inertia in zip(K_range, inertias):
    print(f"k={k}: {inertia:.2f}")

### 5.2 Apply K-Means Clustering

In [ ]:
# Apply K-Means with optimal k=4
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df_features['Cluster'] = kmeans.fit_predict(cluster_scaled)

# Cluster profiles
cluster_summary = df_features.groupby('Cluster').agg({
    'Price ($)': ['mean', 'median', 'std'],
    'Rating': ['mean', 'median', 'std'],
    'Product name': 'count'
}).round(2)

cluster_summary.columns = ['_'.join(col) for col in cluster_summary.columns]
cluster_summary = cluster_summary.rename(columns={'Product name_count': 'Count'})

print("\nCluster Profiles:")
print(cluster_summary)

# Define cluster names
cluster_names = {
    0: 'Premium',
    1: 'Best Value',
    2: 'Standard',
    3: 'Budget'
}
df_features['Cluster_Name'] = df_features['Cluster'].map(cluster_names)

In [ ]:
# Visualize clusters
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df_features['Price ($)'], df_features['Rating'], 
                     c=df_features['Cluster'], cmap='viridis', 
                     alpha=0.6, edgecolors='black', s=50)

# Plot cluster centers
centers = scaler_cluster.inverse_transform(kmeans.cluster_centers_)
plt.scatter(centers[:, 0], centers[:, 1], c='red', s=300, alpha=0.8, 
           edgecolors='black', linewidth=2, marker='X', label='Centroids')

plt.xlabel('Price ($)', fontsize=12)
plt.ylabel('Rating', fontsize=12)
plt.title('Customer Segmentation: Price vs Rating (K-Means, k=4)', fontsize=14)
plt.colorbar(scatter, label='Cluster')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../visualizations/clusters_by_price_rating.png', dpi=300, bbox_inches='tight')
plt.show()

# Distribution by cluster
print("\nCluster Distribution:")
print(df_features['Cluster_Name'].value_counts().sort_index())

## 6. Key Findings & Business Insights

### Summary of Results:

#### 1. Price-Rating Paradox
- **Correlation: -0.31** between price and rating for product forms
- Higher prices do NOT guarantee higher customer satisfaction
- Budget-friendly products (aerosols, foams) achieve excellent ratings

#### 2. Brand Reputation is Key
- **Brand Average Rating** is the strongest predictor (importance: 0.447)
- Brand trust > Price, Platform, or Category
- Consistent quality builds long-term customer satisfaction

#### 3. Platform-Agnostic Quality
- Customer ratings remain consistent across all platforms
- Platform choice doesn't drive satisfaction
- Product quality matters more than where it's sold

#### 4. Market Segmentation
**Four distinct customer segments identified:**

| Cluster | Segment | Avg Price | Avg Rating | Strategy |
|---------|---------|-----------|------------|----------|
| 0 | Premium | $5,490 | 4.16 | High-end positioning |
| 1 | Best Value | $1,493 | 4.31 | **Optimal target** |
| 2 | Standard | $1,404 | 4.09 | Mass market |
| 3 | Budget | $575 | 3.91 | Price-conscious |

#### 5. Model Performance
- **KNN Regressor Test MSE: 0.139** (rating prediction)
- Successfully predicts ratings with 8 features
- GridSearchCV optimal params: k=15, weights=distance, metric=euclidean

### Business Recommendations:

1. **Pricing Strategy:** Don't overprice - focus on value perception
2. **Brand Building:** Invest in consistent product quality across portfolio
3. **Product Development:** Target the "Best Value" segment (mid-tier pricing, high ratings)
4. **Marketing:** Emphasize product quality over platform prestige
5. **Customer Segmentation:** Tailor strategies for each of the 4 market segments

In [ ]:
# Final statistics summary
print("="*60)
print("BEAUTY RETAIL MARKETING ANALYSIS - FINAL SUMMARY")
print("="*60)
print(f"\nDataset Size: {len(df_features):,} products")
print(f"Brands Analyzed: {df_features['Brand'].nunique():,}")
print(f"Platforms: {', '.join(df_features['Website'].unique())}")
print(f"\nPrice Range: ${df_features['Price ($)'].min():.0f} - ${df_features['Price ($)'].max():,.0f}")
print(f"Rating Range: {df_features['Rating'].min():.1f} - {df_features['Rating'].max():.1f}")
print(f"Median Rating: {df_features['Rating'].median():.2f}")
print(f"\nPrice-Rating Correlation: {df_features['Price ($)'].corr(df_features['Rating']):.3f}")
print(f"\nModel Performance (KNN):")
print(f"  - Test MSE: {mse:.4f}")
print(f"  - Test RMSE: {np.sqrt(mse):.4f}")
print(f"  - R² Score: {test_score:.4f}")
print(f"\nCustomer Segments: {optimal_k}")
print(f"\nTop 3 Most Important Features:")
print(importance_df.head(3).to_string(index=False))
print("\n" + "="*60)